In [ ]:
# Load model
# Load testing data
# Model predict
# Compute Inference time

import csv
import sys
import pandas as pd
import os
import glob
import itertools
import numpy as np
from collections import Counter
import math
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

In [ ]:
path = '/content/drive/My Drive/AmericanPics/Test_traces/'
all_files = glob.glob(os.path.join(path, "testing_ssdtrace_expanded_last_50.csv"))
f = all_files[0]  # Change the file name as required
print("File " + str(f))
df = pd.read_csv(f,skiprows =2,header=None,na_values=['-1'], index_col=False,low_memory=False)
cols = ['IO','device_major','device_minor','cpu_id','timestamp','io_sequence','process_id','default_action','lba','unknown','IO_size','deathtime']
df.columns = cols
lba_list = df['lba'].tolist()
deathtime_list = df['deathtime'].tolist()
print("Min LBA in the dataset :", min(lba_list))
print("Max LBA in the dataset :", max(lba_list))
print("Number of IO Accesses :",len(df))
print("Number of unique LBAs : ",len(Counter(lba_list)))


In [ ]:
# Split to train, validate and test
# Finding the value 75th percentile of TimeStamp
import math

training_pt_1 = math.floor((len(df)*0.75)) 

lba_train =df[:training_pt_1]['lba_norm'].tolist()
lba_test = df[training_pt_1+1:]['lba_norm'].tolist()

response_norm_train = df[:training_pt_1]['response_norm'].tolist()
response_norm_test = df[training_pt_1+1:]['response_norm'].tolist()

death_time_norm_train = df[:training_pt_1]['deathtime_norm'].tolist()
death_time_norm_test = df[training_pt_1+1:]['deathtime_norm'].tolist()

lba_low_train = df[:training_pt_1]['lba_low_norm'].tolist()
lba_low_test = df[training_pt_1+1:]['lba_low_norm'].tolist()

lba_high_train = df[:training_pt_1]['lba_high_norm'].tolist()
lba_high_test =  df[training_pt_1+1:]['lba_high_norm'].tolist()

size_norm_train = df[:training_pt_1]['IO_size'].tolist()
size_norm_test = df[training_pt_1+1:]['IO_size'].tolist()

output_train = df[:training_pt_1]['Output_Class'].tolist()
output_test = df[training_pt_1+1:]['Output_Class'].tolist()


In [ ]:
lba_train= np.array(lba_train).reshape(-1,1)
lba_test= np.array(lba_test).reshape(-1,1)

response_norm_train = np.array(response_norm_train).reshape(-1,1)
response_norm_test = np.array(response_norm_test).reshape(-1,1)

death_time_norm_train= np.array(death_time_norm_train).reshape(-1,1)
death_time_norm_test= np.array(death_time_norm_test).reshape(-1,1)

lba_low_train= np.array(lba_low_train).reshape(-1,1)
lba_low_test= np.array(lba_low_test).reshape(-1,1)

lba_high_train= np.array(lba_high_train).reshape(-1,1)
lba_high_test= np.array(lba_high_test).reshape(-1,1)

size_train= np.array(lba_low_train).reshape(-1,1)
size_test= np.array(lba_low_test).reshape(-1,1)

output_train= np.array(output_train).reshape(-1,1)
output_test= np.array(output_test).reshape(-1,1)

In [ ]:
def create_dataset2(dataset, window_size):
    dataX, dataY = [], []
    for i in range(len(dataset) - 2 * window_size):
        a = dataset[i:(i + window_size), 0]
        #print(a)
        dataX.append(a)
        b = dataset[(i + window_size):(i + 2* window_size), 0]
        #print(b)
        dataY.append(b)
    return np.array(dataX), np.array(dataY)

lstm_num_timesteps = 32
    
X_train_lba, y_train_lba = create_dataset2(lba_train, lstm_num_timesteps)
X_test_lba, y_test_lba = create_dataset2(lba_test, lstm_num_timesteps)

X_train_response, y_train_response = create_dataset2(response_norm_train, lstm_num_timesteps)
X_test_response, y_test_response = create_dataset2(response_norm_test, lstm_num_timesteps)

X_train_deathtime, y_train_deathtime = create_dataset2(death_time_norm_train, lstm_num_timesteps)
X_test_deathtime, y_test_deathtime = create_dataset2(death_time_norm_test, lstm_num_timesteps)

X_train_lba_low, y_train_lba_low = create_dataset2(lba_low_train, lstm_num_timesteps)
X_test_lba_low, y_test_lba_low = create_dataset2(lba_low_test, lstm_num_timesteps)

X_train_lba_high, y_train_lba_high = create_dataset2(lba_high_train, lstm_num_timesteps)
X_test_lba_high, y_test_lba_high = create_dataset2(lba_high_test, lstm_num_timesteps)

X_size, y_train_size = create_dataset2(size_train, lstm_num_timesteps)
X_size, y_test_size = create_dataset2(size_test, lstm_num_timesteps)

X_train_output, y_train_output = create_dataset2(output_train, lstm_num_timesteps)
X_test_output, y_test_output = create_dataset2(output_test, lstm_num_timesteps)


In [ ]:
# Create model
# Load model
# Generate inference in batch of 64
# Save predictions
# Save time for predictions